In [ ]:

import torch.optim as optim
from torch.utils.data import DataLoader

from pyswarm import pso  # Ensure 'pyswarm' is installed
import os
import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data import TensorDataset
import torch.optim.lr_scheduler as lr_scheduler
from pyswarm import pso  # Ensure 'pyswarm' is installed

import torch.optim as optim



import matplotlib.pyplot as plt

import torch.optim as optim
from torch.utils.data import DataLoader

from pyswarm import pso  # Ensure you have 'pyswarm' installed for PSO
from sklearn.metrics import mean_squared_error
import os
from sklearn.metrics import mean_absolute_error, r2_score
 # 确保这个文件包含 get_model 函数和必要的模型结构
from sklearn.metrics import mean_absolute_error, r2_score
import os
import csv
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0



# 评估函数
def evaluate_model(model, test_loader, criterion,device):
    model.eval()
    total_loss = 0
    predictions = []
    actuals = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            predictions.extend(outputs.view(-1).cpu().numpy())
            actuals.extend(labels.view(-1).cpu().numpy())

    avg_loss = total_loss / len(test_loader.dataset)
    mae = mean_absolute_error(actuals, predictions)
    mse = mean_squared_error(actuals, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(actuals, predictions)
    nrmse = rmse / np.mean(actuals)

    print(f'Test Loss: {avg_loss:.4f}')
    print(f'MAE: {mae:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'R2: {r2:.4f}')
    print(f'NRMSE: {nrmse:.4f}')
    return avg_loss, mae, rmse, r2, nrmse, actuals, predictions

import torch.nn as nn

# 修改 CustomModel 类，添加 batch_size 检查
class CustomModel(nn.Module):
    def __init__(self, hidden_layers, neurons_per_layer, input_shape, output_shape=1):
        super(CustomModel, self).__init__()
        self.input_shape = input_shape
        self.neurons_per_layer = neurons_per_layer
        self.hidden_layers = hidden_layers
        
        layers = []
        # Input layer
        layers.append(nn.Linear(input_shape, neurons_per_layer))
        layers.append(nn.BatchNorm1d(neurons_per_layer))
        layers.append(nn.ReLU())

        # Hidden layers
        for _ in range(hidden_layers):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())

        # Output layer
        layers.append(nn.Linear(neurons_per_layer, output_shape))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        # 如果 batch_size=1，跳过 BatchNorm
        if x.size(0) == 1:
            layers = []
            layers.append(nn.Linear(self.input_shape, self.neurons_per_layer))
            layers.append(nn.ReLU())
            for _ in range(self.hidden_layers):
                layers.append(nn.Linear(self.neurons_per_layer, self.neurons_per_layer))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(self.neurons_per_layer, 1))
            temp_model = nn.Sequential(*layers)
            return temp_model(x)
        return self.model(x)


# Function to create the model with given parameters
def get_model(hidden_layers, neurons_per_layer, input_shape, output_shape=1):
    """
    根据隐藏层数、每层神经元数量和输入输出形状创建模型实例。

    参数:
    - hidden_layers: int，隐藏层的数量
    - neurons_per_layer: int，每个隐藏层中的神经元数量
    - input_shape: int，输入特征的数量
    - output_shape: int，模型的输出大小，默认为1

    返回:
    - model: CustomModel实例
    """
    model = CustomModel(hidden_layers=hidden_layers,
                        neurons_per_layer=neurons_per_layer,
                        input_shape=input_shape,
                        output_shape=output_shape)
    return model


def train_and_validate(model, train_loader, val_loader, criterion, optimizer, device, model_save_path, fig_name, num_epochs=200, l2_lambda=0.01, pso_params=None):
    train_losses = []
    val_losses = []
    min_val_loss = float('inf')
    output_dir = 'pso_models_and_figs'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    early_stopping = EarlyStopping(patience=10)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            l2_norm = sum(p.pow(2).sum() for p in model.parameters())
            loss += l2_lambda * l2_norm
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item() * inputs.size(0)
        avg_train_loss = total_train_loss / len(train_loader.dataset)
        train_losses.append(avg_train_loss)

        # 验证阶段
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                l2_norm = sum(p.pow(2).sum() for p in model.parameters())
                loss += l2_lambda * l2_norm
                total_val_loss += loss.item() * inputs.size(0)
        avg_val_loss = total_val_loss / len(val_loader.dataset)
        val_losses.append(avg_val_loss)

        # 保存最佳模型并显示PSO搜索的超参数
        if avg_val_loss < min_val_loss:
            min_val_loss = avg_val_loss
            torch.save(model.state_dict(), model_save_path)
            print(f'Saved new best model at epoch {epoch+1} with Val Loss: {avg_val_loss:.4f}')
            print(f'Current PSO parameters: {pso_params}')

        print(f'Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        scheduler.step(avg_val_loss)
        
        early_stopping(avg_val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training and Validation Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f'train_val_loss_{fig_name}.png'))

# 修改 pso_objective 函数，确保 batch_size 不会导致问题
def pso_objective(params):
    batch_size = max(int(params[0]), 2)  # 确保 batch_size 至少为2
    hidden_layers = int(params[1])
    neurons_per_layer = int(params[2])

    model = get_model(hidden_layers=hidden_layers, 
                     input_shape=X_train_sub.shape[1], 
                     neurons_per_layer=neurons_per_layer)
    model = model.to(device)
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
    
    # 在 DataLoader 中使用 drop_last=True，确保最后一个不足的batch被丢弃
    train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                            shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, 
                           shuffle=False, drop_last=True)

    model_save_path = f"pso_best_model_fold{fold}_{batch_size}_{hidden_layers}_{neurons_per_layer}.pth"
    fig_name = f"fold{fold}_loss_fig_{batch_size}_{hidden_layers}_{neurons_per_layer}"
    
    try:
        train_and_validate(model, train_loader, val_loader, criterion, optimizer, device, 
                         model_save_path, fig_name, num_epochs=200, pso_params=params)
        
        model.load_state_dict(torch.load(model_save_path))
        val_loss, *_ = evaluate_model(model, val_loader, criterion, device)
    except RuntimeError as e:
        print(f"Error in training with params {params}: {str(e)}")
        return float('inf')  # 返回一个大值，让PSO避免这个参数组合
        
    return val_loss



In [ ]:
# ... (保持前面的导入和类定义不变直到数据处理部分) ...

import joblib
from sklearn.model_selection import KFold
import pandas as pd

# 数据处理部分
data = pd.read_csv('D:\Antarcatica_GHF\cor_ghf_subsets\World_train_data.csv')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
categorical_cols = ['Rocktype', 'TectReign']
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

X = data.drop(columns=['lon', 'lat', 'heat flow'])
y = data['heat flow']

# 保存 label_encoders
joblib.dump(label_encoders, 'label_encoders.joblib')

# 标准化特征数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 转换为 torch 张量
X_tensor = torch.tensor(X_scaled).float().to(device)  # 确保初始张量在正确设备上
y_tensor = torch.tensor(y.values).float().view(-1, 1).to(device)



# 定义五折交叉验证
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_results = []
all_fold_params = []
all_fold_metrics = []
all_fold_predictions = []  # 存储每折的预测结果
# PSO 搜索超参数
lb = [64, 4, 32]  # batch_size, hidden_layers, neurons_per_layer 下界
ub = [128, 7, 128]  # 上界

# 存储每个折的最佳参数和结果
all_fold_params = []
all_fold_metrics = []
# 用于估计不确定性的多次预测次数
NUM_SAMPLES = 3  # 可以调整此值，越多越精确但计算时间更长
# 修改交叉验证循环中的数据加载部分
for fold, (train_idx, test_idx) in enumerate(kf.split(X_scaled)):
    print(f"\nFold {fold + 1}/5")
    
    X_train_fold = X_tensor[train_idx].to(device)
    y_train_fold = y_tensor[train_idx].to(device)
    X_test_fold = X_tensor[test_idx].to(device)
    y_test_fold = y_tensor[test_idx].to(device)
    
    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(
        X_train_fold, y_train_fold, test_size=0.2, random_state=42
    )

    X_train_sub = torch.tensor(X_train_sub).float().to(device)
    y_train_sub = torch.tensor(y_train_sub).float().to(device)
    X_val_sub = torch.tensor(X_val_sub).float().to(device)
    y_val_sub = torch.tensor(y_val_sub).float().to(device)

    train_dataset = TensorDataset(X_train_sub, y_train_sub)
    val_dataset = TensorDataset(X_val_sub, y_val_sub)
    test_dataset = TensorDataset(X_test_fold, y_test_fold)
    
    # 确保数据集足够大
    if len(train_dataset) < 2 or len(val_dataset) < 2:
        print(f"Warning: Fold {fold + 1} has insufficient data. Skipping...")
        continue
    
    # 修改 pso_objective 函数以使用当前的fold数据
    def pso_objective(params):
        batch_size = max(int(params[0]), 2)
        hidden_layers = int(params[1])
        neurons_per_layer = int(params[2])

        model = get_model(hidden_layers=hidden_layers, 
                         input_shape=X_train_sub.shape[1], 
                         neurons_per_layer=neurons_per_layer).to(device)
        
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.004)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                                shuffle=True, drop_last=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, 
                               shuffle=False, drop_last=True)

        model_save_path = f"pso_best_model_fold{fold}_{batch_size}_{hidden_layers}_{neurons_per_layer}.pth"
        fig_name = f"fold{fold}_loss_fig_{batch_size}_{hidden_layers}_{neurons_per_layer}"
        
        try:
            train_and_validate(model, train_loader, val_loader, criterion, optimizer, device, 
                             model_save_path, fig_name, num_epochs=200, pso_params=params)
            
            model.load_state_dict(torch.load(model_save_path))
            val_loss, *_ = evaluate_model(model, val_loader, criterion, device)
        except RuntimeError as e:
            print(f"Error in training with params {params}: {str(e)}")
            return float('inf')
        
        return val_loss

    # 执行 PSO 优化
    best_params, best_val_loss = pso(pso_objective, lb, ub, swarmsize=10, maxiter=10)
    
    # 保存最佳参数
    best_batch_size = int(best_params[0])
    best_hidden_layers = int(best_params[1])
    best_neurons_per_layer = int(best_params[2])
    
    print(f'Fold {fold + 1} - Optimal Parameters: Batch Size: {best_batch_size}, '
          f'Hidden Layers: {best_hidden_layers}, Neurons/Layer: {best_neurons_per_layer}')
    print(f'Fold {fold + 1} - Best Validation Loss: {best_val_loss:.4f}')
    
    # 使用最佳参数训练最终模型并评估测试集
    best_model = get_model(best_hidden_layers, best_neurons_per_layer, 
                          input_shape=X_train_sub.shape[1]).to(device)
    best_model_save_path = f'pso_best_model_fold{fold}_{best_batch_size}_{best_hidden_layers}_{best_neurons_per_layer}.pth'
    best_model.load_state_dict(torch.load(best_model_save_path))
    
    test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False)
    test_loss, mae, rmse, r2, nrmse, actuals, predictions = evaluate_model(
        best_model, test_loader, torch.nn.MSELoss(), device
    )
    # 计算预测值和标准差（通过多次预测）
    best_model.eval()
    all_fold_preds = []
    with torch.no_grad():
        for _ in range(NUM_SAMPLES):
            fold_preds = []
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                outputs = best_model(inputs)
                fold_preds.extend(outputs.view(-1).cpu().numpy())
            all_fold_preds.append(fold_preds)

    # 计算每折的预测均值和标准差
    all_fold_preds = np.array(all_fold_preds)  # (NUM_SAMPLES, n_test_samples)
    fold_mean_preds = np.mean(all_fold_preds, axis=0)
    fold_std_preds = np.std(all_fold_preds, axis=0)
    
    # 保存每折的预测结果到CSV，包括预测值和标准差
    fold_df = pd.DataFrame({
        'lon': data.iloc[test_idx]['lon'],
        'lat': data.iloc[test_idx]['lat'],
        'actual_heat_flow': actuals,
        'predicted_heat_flow': fold_mean_preds,
        'predicted_std': fold_std_preds
    })

    fold_csv_path = f'fold_{fold + 1}_predictions_with_std.csv'
    fold_df.to_csv(fold_csv_path, index=False)
    print(f"Saved Fold {fold + 1} predictions with std to {fold_csv_path}")

    
    all_fold_predictions.append((fold_mean_preds, fold_std_preds, test_idx))
    all_fold_params.append([best_batch_size, best_hidden_layers, best_neurons_per_layer])
    all_fold_metrics.append({'test_loss': test_loss, 'mae': mae, 'rmse': rmse, 'r2': r2, 'nrmse': nrmse})



# 集成五折预测结果
n_samples = X_scaled.shape[0]
ensemble_preds = np.zeros((5, n_samples))  # 存储每折的预测均值
ensemble_stds = np.zeros((5, n_samples))   # 存储每折的预测标准差
for fold, (fold_preds, fold_stds, test_idx) in enumerate(all_fold_predictions):
    ensemble_preds[fold, test_idx] = fold_preds
    ensemble_stds[fold, test_idx] = fold_stds

# 计算集成均值和标准差
ensemble_mean = np.zeros(n_samples)
ensemble_std = np.zeros(n_samples)
for i in range(n_samples):
    valid_preds = ensemble_preds[:, i][ensemble_preds[:, i] != 0]  # 排除未预测的（0值）
    valid_stds = ensemble_stds[:, i][ensemble_stds[:, i] != 0]
    if len(valid_preds) > 0:
        ensemble_mean[i] = np.mean(valid_preds)
        # 集成标准差：结合模型间方差和模型内方差
        ensemble_std[i] = np.sqrt(np.mean(valid_stds**2) + np.var(valid_preds))

# 保存集成结果
ensemble_df = pd.DataFrame({
    'lon': data['lon'],
    'lat': data['lat'],
    'actual_heat_flow': y,
    'ensemble_predicted_heat_flow': ensemble_mean,
    'ensemble_predicted_std': ensemble_std
})
ensemble_df.to_csv('ensemble_predictions_with_std.csv', index=False)
print("Saved ensemble predictions with std to 'ensemble_predictions_with_std.csv'")

In [ ]:
# 集成测试
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch.nn as nn
import os

# 定义模型类（与训练时相同）
class CustomModel(nn.Module):
    def __init__(self, hidden_layers, neurons_per_layer, input_shape, output_shape=1):
        super(CustomModel, self).__init__()
        self.input_shape = input_shape
        self.neurons_per_layer = neurons_per_layer
        self.hidden_layers = hidden_layers
        
        layers = []
        layers.append(nn.Linear(input_shape, neurons_per_layer))
        layers.append(nn.BatchNorm1d(neurons_per_layer))
        layers.append(nn.ReLU())
        
        for _ in range(hidden_layers):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(neurons_per_layer, output_shape))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        if x.size(0) == 1:  # 如果 batch_size=1，跳过 BatchNorm
            layers = []
            layers.append(nn.Linear(self.input_shape, self.neurons_per_layer))
            layers.append(nn.ReLU())
            for _ in range(self.hidden_layers):
                layers.append(nn.Linear(self.neurons_per_layer, self.neurons_per_layer))
                layers.append(nn.ReLU())
            layers.append(nn.Linear(self.neurons_per_layer, 1))
            temp_model = nn.Sequential(*layers)
            return temp_model(x)
        return self.model(x)

def get_model(hidden_layers, neurons_per_layer, input_shape, output_shape=1):
    return CustomModel(hidden_layers, neurons_per_layer, input_shape, output_shape)

# 假设推理数据集路径为 'Antar_test_data.csv'
inference_data = pd.read_csv('Antar_test_data_with_heatflux.csv')

# 加载保存的 LabelEncoder 和 Scaler
label_encoders = joblib.load('label_encoders.joblib')


# 数据预处理
categorical_cols = ['Rocktype', 'TectReign']
for col in categorical_cols:
    try:
        inference_data[col] = label_encoders[col].transform(inference_data[col])
    except ValueError as e:
        print(f"Warning: Unknown categories in {col}. Filling with default value.")
        inference_data[col] = inference_data[col].apply(
            lambda x: label_encoders[col].transform([x])[0] if x in label_encoders[col].classes_ else -1
        )

# 提取特征
X_new = inference_data.drop(columns=['lon', 'lat', 'heat flow'])
X_new_scaled = scaler.transform(X_new)

# 转换为张量
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_new_tensor = torch.tensor(X_new_scaled).float().to(device)

# 五折模型的参数（根据你的训练结果更新）
fold_params = [
    (107, 7, 123),  # Fold 1: Batch Size: 107, Hidden Layers: 7, Neurons/Layer: 123
    (78, 6, 102),   # Fold 2: Batch Size: 78, Hidden Layers: 6, Neurons/Layer: 102
    (90, 5, 97),    # Fold 3: Batch Size: 90, Hidden Layers: 5, Neurons/Layer: 97
    (126, 6, 79),   # Fold 4: Batch Size: 126, Hidden Layers: 6, Neurons/Layer: 79
    (128, 6, 103),  # Fold 5: Batch Size: 128, Hidden Layers: 6, Neurons/Layer: 103
]

# 创建保存预测的目录
output_dir = 'fold_predictions'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 集成预测并保存每折结果
all_predictions = []
for fold, (batch_size, hidden_layers, neurons) in enumerate(fold_params):
    print(f"Loading and predicting with model for Fold {fold + 1}")
    model = get_model(hidden_layers, neurons, X_new_tensor.shape[1]).to(device)
    model_path = f'pso_best_model_fold{fold}_{batch_size}_{hidden_layers}_{neurons}.pth'
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    loader = DataLoader(TensorDataset(X_new_tensor, torch.zeros_like(X_new_tensor[:, :1])), 
                        batch_size=batch_size, shuffle=False)
    
    fold_predictions = []
    with torch.no_grad():
        for inputs, _ in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            fold_predictions.extend(outputs.view(-1).cpu().numpy())
    all_predictions.append(fold_predictions)
    
    # 保存每折的预测结果到 CSV
    fold_df = inference_data[['lon', 'lat']].copy()
    fold_df['predicted_heat_flow'] = fold_predictions
    fold_csv_path = os.path.join(output_dir, f'fold_{fold + 1}_inference_predictions.csv')
    fold_df.to_csv(fold_csv_path, index=False)
    print(f"Saved Fold {fold + 1} predictions to {fold_csv_path}")

# 计算集成预测的均值和标准差
predictions = np.mean(all_predictions, axis=0)
y_std = np.std(all_predictions, axis=0)

# 将预测均值和标准差添加到数据框
inference_data['ensemble_predicted_heat_flow'] = predictions
inference_data['ensemble_predicted_std'] = y_std

# 评估有真实值的行
if 'heat flow' in inference_data.columns:
    valid_indices = inference_data['heat flow'].notna()
    y_true = inference_data.loc[valid_indices, 'heat flow'].values
    y_pred = inference_data.loc[valid_indices, 'ensemble_predicted_heat_flow'].values
    
    if len(y_true) > 0:
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)
        print(f"\nPerformance on {len(y_true)} rows with true values (Ensemble):")
        print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")
    else:
        print("No valid true values to evaluate.")

# 保存集成结果
output_file = 'ensemble_inference_predictions_with_std.csv'
inference_data.to_csv(output_file, index=False)
print(f"Ensemble predictions saved to {output_file}")

# 显示前几行结果
print("\nSample of the updated dataset:")
print(inference_data[['lon', 'lat', 'heat flow', 'ensemble_predicted_heat_flow', 'ensemble_predicted_std']].head())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, TensorDataset

# Import Uncertainty Toolbox
import uncertainty_toolbox as uct

# Set random seed (optional, since we’re using real data)
seed = 111
np.random.seed(seed)
torch.manual_seed(seed)

# Set plot style (as in the tutorial)
uct.viz.set_style()
uct.viz.update_rc('figure.dpi', 130)
uct.viz.update_rc('text.usetex', False)

# Load your CSV file
df = pd.read_csv('ensemble_inference_predictions_with_std4.csv')  # Replace with your CSV file path

# Extract data
y_true = df['heat flow'].values  # Observed values (equivalent to te_y in the tutorial)
pred_mean = df['ensemble_predicted_heat_flow'].values  # Predicted means (equivalent to pred_mean)
pred_std = df['ensemble_predicted_std'].values  # Predicted standard deviations (equivalent to pred_std)

# Ensure data is clean (no NaN or infinite values)
mask = ~np.isnan(y_true) & ~np.isnan(pred_mean) & ~np.isnan(pred_std)
y_true = y_true[mask]
pred_mean = pred_mean[mask]
pred_std = pred_std[mask]

# Modify x-axis for the first plot: Sort data by predicted_heat_flow to order from smallest to largest
sort_idx = np.argsort(pred_mean)  # Sort by predicted_heat_flow
x_sorted = np.arange(len(pred_mean))[sort_idx]  # Create an index (0 to n-1) for the sorted order
y_true_sorted = y_true[sort_idx]
pred_mean_sorted = pred_mean[sort_idx]
pred_std_sorted = pred_std[sort_idx]

# 1. Create a figure with three subplots in a row
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))  # 1 row, 3 columns, adjusted figure size

# 1. Plot Confidence Bands on Test Data (using sorted order instead of lon)
uct.viz.plot_xy(pred_mean_sorted, pred_std_sorted, y_true_sorted, x_sorted, ax=ax1)  # Use sorted index as x-axis
ax1.set_title('Confidence Bands')
ax1.set_xlabel('Index (Ordered by Observed Heat Flux)')
ax1.set_ylabel('Heat Flux')

# 2. Plot Ordered Prediction Intervals
uct.viz.plot_intervals_ordered(pred_mean, pred_std, y_true, ax=ax2)
ax2.set_title('Ordered Prediction Intervals')

# 3. Plot Average Calibration
uct.viz.plot_calibration(pred_mean, pred_std, y_true, ax=ax3)
ax3.set_title('Average Calibration')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

# 4. Plot Adversarial Group Calibration (optional, as in the tutorial)
plt.figure(figsize=(5, 3))
uct.viz.plot_adversarial_group_calibration(pred_mean, pred_std, y_true)
plt.title('Adversarial Group Calibration')
plt.show()

# 5. Get All Metrics (optional, for evaluation)
pnn_metrics = uct.metrics.get_all_metrics(pred_mean, pred_std, y_true)
print("Uncertainty Metrics:", pnn_metrics)

# Optional: Recalibrate Predictive Uncertainties (as in the tutorial, if you want to improve calibration)
# This requires a separate recalibration dataset, which you’d need to generate or provide.
# For simplicity, we’ll skip this step unless you provide additional data for recalibration.

# Optional: Visualize 95% Prediction Intervals (you can adapt this from the tutorial)
# Note: We’ll use the original unsorted x (lon) here, but you can modify it similarly if needed
orig_bounds = uct.metrics_calibration.get_prediction_interval(pred_mean, pred_std, 0.95, None)
x = df['lon'].values[mask]  # Reusing lon as x for consistency with the original plot
plt.figure(figsize=(5, 3))
plt.fill_between(x, orig_bounds.lower, orig_bounds.upper, alpha=0.6, label='Original 95% Interval')
plt.scatter(x, y_true, color='black', alpha=0.5, label='Observed Heat Flux')
plt.xlabel('Longitude (lon)')
plt.ylabel('Heat Flux')
plt.title('95% Prediction Interval (Spatial)')
plt.legend()
plt.show()

# Optional: Recalibrate by scaling standard deviations (as in the tutorial)
# This would require a separate recalibration dataset, which you’d need to provide.
# For now, we’ll skip this unless you have additional data.